In [87]:
import pandas as pd
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import os

In [88]:
# duplicates
# missing values
# outliers

In [89]:
# df = pd.concat(map(pd.read_csv, glob.glob("raw_data/ust_aq_timeseries_*.csv")))

In [90]:
df = pd.read_csv("raw_data/ust_aq_timeseries_2018.csv")

In [91]:
print(len(df)-len(df.drop_duplicates()))
df = df.drop_duplicates()

0


In [92]:
# df =df[df['station_name'] == 'Eyjavellir, Reykjanesbær']

In [93]:
# MIssing hourly readings?

In [94]:
pivot_df = pd.pivot_table(df, values='the_value', index=['station_name', 'endtime'],
                       columns=['pollutantnotation'], aggfunc=lambda x: 'n/a' if x.empty else x.iloc[0]).rename_axis(columns=None).reset_index()

In [95]:
pivot_df.head()

,station_name,endtime,H2S,NO,NO2,NOX as NO2,PM1,PM10,PM2.5,SO2
0,Akureyri Strandgata (við Hof),2018-01-01 00:00:00,NaN,17.1910,50.5742,73.8981,NaN,32.3,NaN,2.08356
1,Akureyri Strandgata (við Hof),2018-01-01 01:00:00,NaN,95.9104,83.8490,229.2860,NaN,136.0,NaN,9.17334
2,Akureyri Strandgata (við Hof),2018-01-01 02:00:00,NaN,53.6729,75.3962,156.4620,NaN,884.0,NaN,3.94667
3,Akureyri Strandgata (við Hof),2018-01-01 03:00:00,NaN,22.4312,64.4277,95.4120,NaN,61.0,NaN,2.35111
4,Akureyri Strandgata (við Hof),2018-01-01 04:00:00,NaN,15.4146,43.4120,62.2421,NaN,22.0,NaN,1.64445


In [96]:
result = pivot_df.groupby("station_name")["endtime"].nunique().reset_index(name="unique_count_endtime")
result.head()

,station_name,unique_count_endtime
0,Akureyri Strandgata (við Hof),7912
1,Akureyri Tryggvabraut,7912
2,"Eyjavellir, Reykjanesbær",8706
3,Grindavík Nesvegur,8761
4,Grundartangi Gröf,8760


In [97]:
# Convert the 'time' column to datetime format
pivot_df['endtime'] = pd.to_datetime(pivot_df['endtime'])

# Set 'time' column as the DataFrame's index
pivot_df.set_index('endtime', inplace=True)

# Resample to create complete hourly time range for each station
# The 'ffill' method fills missing values using the last available value (forward fill)
# The 'fillna' method replaces any remaining missing values with 'N/A'
hourly_range = pd.date_range(start=pivot_df.index.min(), end=pivot_df.index.max(), freq='1H')
result = pivot_df.groupby('station_name').apply(lambda x: x.reindex(hourly_range, method='ffill')).fillna('N/A')

result.head()

station_name  \
station_name                                                                       
Akureyri Strandgata (við Hof) 2018-01-01 00:00:00  Akureyri Strandgata (við Hof)   
                              2018-01-01 01:00:00  Akureyri Strandgata (við Hof)   
                              2018-01-01 02:00:00  Akureyri Strandgata (við Hof)   
                              2018-01-01 03:00:00  Akureyri Strandgata (við Hof)   
                              2018-01-01 04:00:00  Akureyri Strandgata (við Hof)   

                                                   H2S       NO      NO2  \
station_name                                                               
Akureyri Strandgata (við Hof) 2018-01-01 00:00:00  N/A   17.191  50.5742   
                              2018-01-01 01:00:00  N/A  95.9104   83.849   
                              2018-01-01 02:00:00  N/A  53.6729  75.3962   
                              2018-01-01 03:00:00  N/A  22.4312  64.4277   
                              2018-01-01 04:00:00  N/A  15.4146   43.412   

                                                  NOX as NO2  PM1   PM10  \
station_name                                                               
Akureyri Strandgata (við Hof) 2018-01-01 00:00:00    73.8981  N/A   32.3   
                              2018-01-01 01:00:00    229.286  N/A  136.0   
                              2018-01-01 02:00:00    156.462  N/A  884.0   
                              2018-01-01 03:00:00     95.412  N/A   61.0   
                              2018-01-01 04:00:00    62.2421  N/A   22.0   

                                                  PM2.5      SO2  
station_name                                                      
Akureyri Strandgata (við Hof) 2018-01-01 00:00:00   N/A  2.08356  
                              2018-01-01 01:00:00   N/A  9.17334  
                              2018-01-01 02:00:00   N/A  3.94667  
                              2018-01-01 03:00:00   N/A  2.35111  
                              2018-01-01 04:00:00   N/A  1.64445

In [98]:
# Reset the MultiIndex
reset_df = result.reset_index(level='station_name', drop=True)

In [99]:
df = reset_df.reset_index().rename(columns={'index': 'endtime'})
df.head()

,endtime,station_name,H2S,NO,NO2,NOX as NO2,PM1,PM10,PM2.5,SO2
0,2018-01-01 00:00:00,Akureyri Strandgata (við Hof),N/A,17.191,50.5742,73.8981,N/A,32.3,N/A,2.08356
1,2018-01-01 01:00:00,Akureyri Strandgata (við Hof),N/A,95.9104,83.849,229.286,N/A,136.0,N/A,9.17334
2,2018-01-01 02:00:00,Akureyri Strandgata (við Hof),N/A,53.6729,75.3962,156.462,N/A,884.0,N/A,3.94667
3,2018-01-01 03:00:00,Akureyri Strandgata (við Hof),N/A,22.4312,64.4277,95.412,N/A,61.0,N/A,2.35111
4,2018-01-01 04:00:00,Akureyri Strandgata (við Hof),N/A,15.4146,43.412,62.2421,N/A,22.0,N/A,1.64445


In [100]:
# Find rows with NaN in column 'B'
na_rows = df[df['station_name'] == 'N/A']

# Get the row above and below each NaN row
na_indices = na_rows.index
na_rows_and_neighbors = df.iloc[na_indices - 1].append(na_rows).append(df.iloc[na_indices + 1])

na_rows_and_neighbors

,endtime,station_name,H2S,NO,NO2,NOX as NO2,PM1,PM10,PM2.5,SO2
17521,2019-01-01 00:00:00,Akureyri Tryggvabraut,N/A,1.34583,6.0629,7.43397,N/A,22.0,N/A,0.844445
192741,2019-01-01 00:00:00,Reykjanes Mánagrund,N/A,N/A,N/A,N/A,N/A,N/A,N/A,1.92498
17522,2018-01-01 00:00:00,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
192742,2018-01-01 00:00:00,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
17523,2018-01-01 01:00:00,"Eyjavellir, Reykjanesbær",N/A,6.67708,11.8365,19.8648,N/A,N/A,N/A,N/A
192743,2018-01-01 01:00:00,Reykjavik Grensas,8.28682,198.096,92.3826,384.22,N/A,1457.2,1141.05,28.2033


In [101]:
df = df[df['station_name'] != 'N/A']

In [102]:
df.groupby('station_name').count()

,endtime,H2S,NO,NO2,NOX as NO2,PM1,PM10,PM2.5,SO2
station_name,,,,,,,,,
Akureyri Strandgata (við Hof),8761,8761,8761,8761,8761,8761,8761,8761,8761
Akureyri Tryggvabraut,8761,8761,8761,8761,8761,8761,8761,8761,8761
"Eyjavellir, Reykjanesbær",8760,8760,8760,8760,8760,8760,8760,8760,8760
Grindavík Nesvegur,8761,8761,8761,8761,8761,8761,8761,8761,8761
Grundartangi Gröf,8761,8761,8761,8761,8761,8761,8761,8761,8761
Grundartangi Kríuvarða,8761,8761,8761,8761,8761,8761,8761,8761,8761
Grundartangi Melahverfi,8761,8761,8761,8761,8761,8761,8761,8761,8761
Hafnarfjörður Hvaleyrarholt,8761,8761,8761,8761,8761,8761,8761,8761,8761
Hafnarfjörður Norðurhella,8761,8761,8761,8761,8761,8761,8761,8761,8761


In [16]:
lknds

NameError: name 'lknds' is not defined

In [ ]:

# Group by 'Group_Column' and check the number of NaN values in other columns for each group
na_count_per_group = df.groupby('station_name').apply(lambda x: x.isna().sum())

# Alternatively, you can use the aggregate function `sum()` instead of `apply()` for better performance:
# na_count_per_group = df.groupby('Group_Column').isna().sum()

# Reset the index to get a more readable output
na_count_per_group = na_count_per_group.reset_index()

# Pivot the DataFrame to have 'Group_Column' as index and columns with their own column names
na_count_pivot = na_count_per_group.pivot_table(index='station_name', columns='index', values=0)

# Rename the columns for better clarity
na_count_pivot.columns.name = None
na_count_pivot.columns = ['{}_na_count'.format(col) for col in na_count_pivot.columns]


In [ ]:
# result = df.groupby(['station_name', 'pollutantnotation'])['the_value'].apply(lambda x: x.isna().sum()).reset_index(name='NAs_Count')

In [ ]:
# create new date and time columns

In [ ]:
# Convert column to datetime type
df['endtime'] = pd.to_datetime(df['endtime'])  

# Create new columns for date and time
df['date'] = df['endtime'].dt.date
df['hour'] = df['endtime'].dt.time

In [ ]:
df = df.sort_values(by=['date', 'hour'])

In [ ]:
df.tail(50)